# KWEST Matching Program
Code to match Kellogg students to a KWEST trip based on submitted preferences.

#### Steps
1. Change `INPUT_FPATH` and `OUTPUT_FPATH` to location of input trip preferences excel file and preferred output location for CSV file of trip rosters
1. Run all code cells

#### Suggestions
- Require all students to submit 10 trip preferences - this increased likelihood of optimal trip matches

### Set `INPUT_FPATH` and `OUTPUT_FPATH`

In [50]:
import os
INPUT_FPATH = os.path.join(os.getcwd(),'data.xlsx')
OUTPUT_FPATH = os.path.join(os.getcwd(),'output.csv')

### Import

##### Packages

In [51]:
from random import shuffle
from math import ceil
from itertools import product
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from matching.games import HospitalResident

##### Data

In [95]:
df_input = pd.read_excel(INPUT_FPATH,dtype=str)

##### Globals

In [96]:
attr_cols = ['Netid','Gender','Program','Passport Country']#,'Date of Birth']
vote_cols = ['Vote'+str(i+1) for i in range(10)]
keep_cols = attr_cols + vote_cols + ['ParentNetid']

### Preference Prediction
For students who set less than 10 preferences, predict the remaining slots using KNN algorithm.

In [143]:
netid_votes = list(product(df_input['Netid'].unique().tolist(), [(i+1) for i in range(10)]))

def _sig_other_indicator(row):
    if row['Sig Other Coming'].lower() == 't':
        return 1
    else:
        return 0
df_input['Significant Other'] = df_input.apply(lambda row: _sig_other_indicator(row), axis=1)

df_votes = pd.melt(df_input,id_vars=attr_cols+['Significant Other'],value_vars=vote_cols,var_name='Vote',value_name='Trip').dropna()
df_votes['Gender'] = df_votes['Gender'].apply(lambda x: 1 if x=='F' else 0)
df_votes['Program'] = df_votes['Program'].apply(lambda x: 1 if x=='2YMBA' else 0)
df_votes['Passport Country'] = df_votes['Passport Country'].apply(lambda x: 1 if x=='UNITED STATES' else 0)
df_votes['Vote'] = df_votes['Vote'].str.replace('Vote','').astype(int)

df_blank = pd.DataFrame(netid_votes,columns=['Netid','Vote'])\
    .merge(df_votes[['Netid','Vote','Trip']],on=['Netid','Vote'],how='left')\
    .merge(df_votes[['Netid','Gender','Program','Passport Country','Significant Other']].drop_duplicates(),on=['Netid'],how='inner')
df_blank = df_blank.loc[df_blank.Trip.isnull()].drop(columns='Trip')

In [144]:
vote_dfs = {}
for vote in vote_cols:
    vote_num = int(vote.replace('Vote',''))
    vote_dfs[vote] = {
        'fit': {
            'x': df_votes.loc[df_votes.Vote==vote_num].set_index('Netid').drop(columns=['Trip','Vote']),
            'y': df_votes.loc[df_votes.Vote==vote_num].set_index('Netid').Trip,
        },
        'predict': {
            'x': df_blank.loc[df_blank.Vote==vote_num].set_index('Netid').drop(columns=['Vote']),
            'y': None,
        },
    }

In [145]:
for vote in vote_cols:
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(vote_dfs[vote]['fit']['x'], vote_dfs[vote]['fit']['y'])
    if vote_dfs[vote]['predict']['x'].shape[0]>1:
        vote_dfs[vote]['predict']['y'] = knn.predict(vote_dfs[vote]['predict']['x'])

In [170]:
predict

,Vote2
0,200341 - Brazil
1,200337 - Amalfi Coast
2,200364 - Peru
3,200353 - Hawaii
4,200364 - Peru
5,200337 - Amalfi Coast


In [172]:
vote_dfs[vote]['predict']['y']

array(['200341 - Brazil', '200337 - Amalfi Coast', '200364 - Peru',
       '200353 - Hawaii', '200364 - Peru', '200337 - Amalfi Coast'],
      dtype=object)

In [168]:
df_predictions = pd.DataFrame(df_input['Netid'].unique().tolist(),columns=['Netid']).set_index('Netid')

for vote in vote_cols:
    print(vote)
    fit = pd.DataFrame(vote_dfs[vote]['fit']['y'])
    if fit.shape[0] > 1:
        fit.columns = [vote]
        df_predictions = df_predictions.join(fit,how='left')
    
    predict = pd.DataFrame(vote_dfs[vote]['predict']['y'])
    if predict.shape[0] > 1:
        predict.columns = [vote]
        df_predictions = df_predictions.join(predict,how='left')
        

Vote1
Vote2


ValueError: columns overlap but no suffix specified: Index(['Vote2'], dtype='object')

In [169]:
df_predictions

,Vote1,Vote2
Netid,,
1,200370 - South of France,NaN
2,200372 - Spanish Mediterranean,200352 - Greek Islands
3,200367 - Slovenia,200364 - Peru
4,200361 - Mystery,200369 - South Korea
5,200360 - Morocco,200364 - Peru
...,...,...
565,200337 - Amalfi Coast,200371 - Southern Spain
566,200370 - South of France,200337 - Amalfi Coast
567,200360 - Morocco,200340 - Belize


In [157]:
df_predictions

""
Netid
1
2
3
4
5
...
565
566
567


### Data Manipulation

##### Add Rows for JVs
In order to use the Hospital-Resident matching algorithm, the data needs a single row for each individual attending a trip. Data rows for JVs are created by using JV attributes (gender, nationality, etc.) and copying their partner's trip preferences.

In [85]:
jv_rows = []
for idx,row in df_input.loc[(df_input['Sig Other Coming'] == 'T') & (df_input['Sig Other Kellogg Student'] != 'T')].iterrows():
    jv_rows.append({
        'Netid': str(row['Netid'])+'JV',
        'ParentNetid': str(row['Netid']),
        'Gender': row['Significant Gender'],
        'Program': 'JV',
        'Passport Country': row['Significant Passport Country'],
        'Vote1': row['Vote1'],
        'Vote2': row['Vote2'],
        'Vote3': row['Vote3'],
        'Vote4': row['Vote4'],
        'Vote5': row['Vote5'],
        'Vote6': row['Vote6'],
        'Vote7': row['Vote7'],
        'Vote8': row['Vote8'],
        'Vote9': row['Vote9'],
        'Vote10': row['Vote10'],
    })
    
df_input = pd.concat([df_input,pd.DataFrame(jv_rows)],sort=False,ignore_index=True)[keep_cols]

##### Create Field Groupings
- Program (2YMVA vs Other)
- Nationality (US vs Other)
- Gender (M vs Other)

In [86]:
# groupings for program, nationality, gender
df_input['Program'] = df_input['Program'].apply(lambda x: x if x in ['2YMBA'] else 'OTHER') # 1YMBA
df_input['Passport Country'] = df_input['Passport Country'].apply(lambda x: x if x in ['UNITED STATES'] else 'OTHER')
# df_input['Gender'] = df_input['Gender'].apply(lambda x: '1' if x in ['M'] else '0') # 1YMBA

##### Calculate Demographic Constraint Ranges
- Constraints: gender, program, nationality

In [37]:
# # demographic constraint distribution range calculations
# margin = 0.15
# # constraints
# dist_female = round(df_input.groupby('Gender')['Netid'].count()/len(df_input),4)['F']
# rng_female = [round(dist_female-margin,2),round(dist_female+margin,2)]
# print(dist_female)
# print(rng_female)
# dist_2y = round(df_input.groupby('Program')['Netid'].count()/len(df_input),4)['2YMBA']
# rng_2y = [round(dist_2y-margin,2),round(dist_2y+margin,2)]
# print(dist_2y)
# print(rng_2y)
# dist_amer = round(df_input.groupby('Passport Country')['Netid'].count()/len(df_input),4)['UNITED STATES']
# # rng_nationality = [round(dist_amer-0.25,2),round(dist_amer+0.25,2)]
# rng_amer = [0.1,0.9]
# print(dist_amer)
# print(rng_amer)

##### Create Unique Trip / NetID Lists

In [53]:
trips_unique = [df_input[i].tolist() for i in votes]
trips_unique = [t for trip in trips_unique for t in trip if str(t) != 'nan']
trips_unique = set(trips_unique)
netid_unique = [str(i) for i in df_input['Netid'].unique()]

##### Select Available Trips by Popularity

In [62]:
trip_cap = 18
extra_trips = 2
trips_needed = ceil(len(netid_unique)/trip_cap)
trip_votes = pd.melt(df_input[['Netid']+votes],id_vars=['Netid'],value_vars=votes,var_name='Vote',value_name='Trip').groupby('Trip')['Netid']
trips_final = trip_votes.count().sort_values(ascending=False)[:trips_needed].index.tolist()
trip_capacity = {trip:trip_cap for trip in trips_final}
_trips = trip_votes.apply(list).loc[_trips.index.isin(trips_final)].to_dict()

##### Student Preferences
- Outputs a list of students who didn't choose any of the most popular trips

In [63]:
student_preferences = {}
no_preference_students = []
for idx,row in df_input.iterrows():
    prefs = [i for i in row[votes].values.tolist() if (str(i) != 'nan') & (i in trips_final)]
    if prefs != []:
        student_preferences[str(row['Netid'])] = prefs
    else:
        no_preference_students.append(str(row['Netid']))
no_preference_students

[]

### Matching Program

In [64]:
iterations = 1000
matches = {}
top_match = (None,100)

for i in range(iterations):

    trip_preferences = {}
    for trip,netids in _trips.items():
        _netids = [str(i) for i in netids]
        shuffle(_netids)
        # put a JV directly after partner - max chance of assigned together
        for netid in _netids:
            if netid[-2:]=='JV':
                _netids.insert(
                    # in the spot directly after student partner
                    _netids.index(netid[:-2])+1,
                    # insert the jv netid
                    _netids.pop(_netids.index(netid))
                )
        trip_preferences[trip] = _netids

    game = HospitalResident.create_from_dictionaries(
        student_preferences,
        trip_preferences,
        trip_capacity
    )

    match = game.solve(optimal='resident')

    matched_students = []
    for _, students in match.items():
        for student in students:
            matched_students.append(student.name)

    unmatched_students = set(student_preferences.keys()) - set(matched_students)

    if len(unmatched_students) == 0:
        matches[str(i)] = {'match': match,'jv':[]}
        issues = 0
        trip_num = 0
        try:
            for trip,students in match.items():
                trip_num+=1
                students = [str(i.name) for i in students]
                s = len(students)
                f = 0
                am = 0
                y2 = 0
                for student in students:
                    # jvs with partners
                    if student[-2:]=='JV':
                        if str(student[:-2]) not in students:
                            issues+=1
                            matches[str(i)]['jv'].append(student)
                    # demographic info
                    demo = df_input.loc[df_input['Netid'].apply(str)==student]
                    if demo['Gender'].values[0] == 'F':
                        f+=1
                    if demo['Passport Country'].values[0] == 'UNITED STATES':
                        am+=1
                    if demo['Program'].values[0] == '2YMBA':
                        y2+=1

            
            matches[str(i)]['issues'] = issues
            if issues < top_match[1]:
                top_match = (str(i),issues)

        except Exception as e:
            # raise
            print('match',i+1,'trip',trip_num,'failed:',e)
            continue

run out of iterations


In [13]:
for k,v in matches.items():
    print(k,'issues:',v['issues'])

54 issues: 9
55 issues: 11
122 issues: 11
193 issues: 14
194 issues: 14
200 issues: 15
254 issues: 14
256 issues: 17
260 issues: 16
354 issues: 17
389 issues: 16
407 issues: 10
448 issues: 13
480 issues: 12
506 issues: 12
597 issues: 11
677 issues: 11
682 issues: 15
979 issues: 11


In [14]:
top_match

('54', 9)

In [15]:
for k,v in matches[top_match[0]]['match'].items():
    print(k,': ',len(v))

200336 - Alaska :  20
200337 - Amalfi Coast :  20
200338 - Amazing Race :  20
200339 - Bali :  20
200340 - Belize :  20
200341 - Brazil :  20
200342 - Budapest / Vienna :  20
200343 - Bulgaria :  20
200344 - Canary Islands :  20
200345 - Chile :  20
200346 - Colombia :  20
200347 - Crete :  20
200348 - Croatia :  20
200349 - Cyprus :  20
200351 - Finland / Estonia :  20
200352 - Greek Islands :  20
200353 - Hawaii :  18
200354 - Hong Kong / Taiwan :  15
200355 - Israel :  20
200356 - Italian Adriatic :  20
200357 - Jamaica :  17
200358 - Malta / Sicily :  20
200359 - Montenegro :  20
200360 - Morocco :  20
200361 - Mystery :  20
200362 - New Zealand :  20
200363 - Northern Spain :  20
200364 - Peru :  20
200365 - Portugal :  20
200366 - Romania :  18
200367 - Slovenia :  20
200368 - South Africa :  20
200370 - South of France :  20
200371 - Southern Spain :  20
200372 - Spanish Mediterranean :  20
200374 - Vietnam :  14


In [16]:
for k,v in matches[top_match[0]]['match'].items():
    print(k,': ',len(v),' ',v)

200336 - Alaska :  20   [99, 99JV, 511, 247, 567, 567JV, 423, 423JV, 330, 407, 266, 208, 407JV, 345, 434, 434JV, 201, 524, 223, 468]
200337 - Amalfi Coast :  20   [118, 39, 39JV, 28, 252, 252JV, 443, 260, 484, 450, 450JV, 347, 456, 104, 44, 6, 428, 565, 319, 162]
200338 - Amazing Race :  20   [30, 30JV, 339, 115, 115JV, 316, 418, 387, 516, 371, 445, 400, 126, 92, 92JV, 410, 461, 45, 61, 85]
200339 - Bali :  20   [81, 373, 425, 315, 546, 546JV, 378, 378JV, 156, 529, 103, 514, 514JV, 242, 302, 227, 170, 170JV, 139, 116]
200340 - Belize :  20   [18, 7, 7JV, 140, 140JV, 495, 90, 241, 241JV, 182, 221, 17, 256, 527, 482, 475, 297, 37, 214, 214JV]
200341 - Brazil :  20   [232, 229, 369, 46, 206, 338, 338JV, 171, 48, 82, 382, 55, 57, 362, 356, 363, 363JV, 379, 168, 168JV]
200342 - Budapest / Vienna :  20   [14, 286, 536, 526, 419, 568, 185, 185JV, 296, 153, 153JV, 132, 132JV, 476, 183, 493, 493JV, 518, 518JV, 250]
200343 - Bulgaria :  20   [121, 325, 33, 11, 11JV, 367, 467, 486, 486JV, 322, 32

In [17]:
print(matches[top_match[0]]['jv'])

['251JV', '385JV', '79JV', '484JV', '377JV', '147JV', '411JV', '52JV', '396JV']


In [18]:
trip_match = {}
for trip,students in matches[top_match[0]]['match'].items():
    trip_name = trip.name.strip()
    trip_match[trip_name] = []
    for student in students:
        trip_match[trip_name].append(student.name.strip())

student_match = {}
for trip,students in trip_match.items():
    for student in students:
        student_match[str(student)] = trip

jv_mismatches = matches[top_match[0]]['jv']

In [29]:
jv_mismatches

['251JV', '385JV', '79JV', '484JV', '377JV', '147JV', '411JV', '52JV', '396JV']

In [34]:
rematched = []
for jv in jv_mismatches:
    print('mismatched jv',jv)
    jv_trip = student_match[jv]
    partner_trip = student_match[str(jv[:-2])]
    print('jv trip',jv_trip)
    print('partner trip',partner_trip)
    print('partner trip students',trip_match[partner_trip])
    for alt_student in trip_match[partner_trip]:
        if str(jv[:-2]) == alt_student:
            continue
        if (
            (jv_trip in student_preferences[str(alt_student)]) # if mismatched jv's trip is in the alt students preference list
            and (alt_student+'JV' not in trip_match[partner_trip]) # AND alt student doesn't have JV
            and (alt_student[-2:] != 'JV') # AND alt student isn't a JV
        ):
            print('alt student',alt_student)
            print('alt student prefs',student_preferences[str(alt_student)])
            print('match')
            rematched.append(jv)
            break
            # raise

mismatched jv 251JV
jv trip 200346 - Colombia
partner trip 200351 - Finland / Estonia
partner trip students ['569', '569JV', '287', '538', '83', '105', '436', '161', '161JV', '76', '551', '251', '333', '553', '351', '72', '561', '561JV', '84', '176']
alt student 287
alt student prefs ['200344 - Canary Islands', '200341 - Brazil', '200351 - Finland / Estonia', '200365 - Portugal', '200343 - Bulgaria', '200367 - Slovenia', '200357 - Jamaica', '200361 - Mystery', '200338 - Amazing Race', '200346 - Colombia']
match
mismatched jv 385JV
jv trip 200349 - Cyprus
partner trip 200370 - South of France
partner trip students ['312', '416', '340', '111', '442', '109', '109JV', '187', '463', '435', '1', '1JV', '487', '385', '292', '564', '564JV', '566', '566JV', '455']
alt student 340
alt student prefs ['200370 - South of France', '200365 - Portugal', '200347 - Crete', '200352 - Greek Islands', '200367 - Slovenia', '200342 - Budapest / Vienna', '200349 - Cyprus', '200356 - Italian Adriatic', '200358

In [35]:
jv_mismatches

['251JV', '385JV', '79JV', '484JV', '377JV', '147JV', '411JV', '52JV', '396JV']

In [36]:
rematched

['251JV', '385JV', '79JV', '484JV', '377JV', '147JV', '411JV', '52JV', '396JV']